In [1]:
import parselmouth
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import pandas as pd
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

sns.set() # Use seaborn's default style to make attractive graphs

In [ ]:
def create_spectrogram(path,id):
    dynamic_range=50
    sound = parselmouth.Sound(path)
    sound.pre_emphasize()
    spectrogram = sound.to_spectrogram(window_length=0.05, 
                                   maximum_frequency=5500)
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    
    fig = plt.figure(figsize=(16,12))
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0,
            hspace = 0, wspace = 0)
    plt.margins(0,0)
    fig.canvas.draw()
    #numpy = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    
    
    return plt

In [ ]:
sex_list = []
healthy_list = []


meta = pd.read_excel('../bd/SVD/META/SVD.xls', sheet_name='SVD')
ids = meta['ID'].tolist()
healthy = meta['Healthy'].tolist()
sex = meta['Sex'].tolist()

In [ ]:
for wave_file in glob.glob("../bd/SVD/BD/PHRASE/*.wav"):
    x = wave_file.replace("../bd/SVD/BD/PHRASE", "")
    id = x.replace("-phrase.wav", "")
    id = id.replace("\\", "")
    placeId = ids.index(int(id))
    print(id)
    plt = create_spectrogram(wave_file,id)
    plt.savefig("SVD/spectrograms_PHRASE/women/"+id+".jpg") if sex[placeId]=="m" else plt.savefig("SVD/spectrograms_PHRASE/men/"+id+".jpg")
    plt.close()

In [2]:
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(1152, 864, 3),
    include_top=False)

# Freeze base model
base_model.trainable = False

# Create inputs with correct shape
inputs = keras.Input(shape=(1152, 864, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x =  keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(1, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

# Important to use binary crossentropy and binary accuracy as we now have a binary classification problem
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

# Create a data generator
datagen_train = ImageDataGenerator(
#    samplewise_center=True,  # set each sample mean to 0
#    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#    zoom_range=0.1,  # Randomly zoom image
#    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#    horizontal_flip=True,  # randomly flip images
#    vertical_flip=False,
)  # we don't expect Bo to be upside-down so we will not flip vertically

# No need to augment validation data
datagen_valid = ImageDataGenerator(samplewise_center=True)

# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "SVD/spectrograms_PHRASE/train/",
    target_size=(1152, 864),
    color_mode="rgb",
    class_mode="binary",
    batch_size=1,
)

# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "SVD/spectrograms_PHRASE/valid/",
    target_size=(1152, 864),
    color_mode="rgb",
    class_mode="binary",
    batch_size=1,
)

model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 837 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/10
12/12 [==============================] - 30s 3s/step - loss: 0.9799 - binary_accuracy: 0.3333 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 2/10
12/12 [==============================] - 35s 3s/step - loss: 0.8133 - binary_accuracy: 0.5000 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 3/10
12/12 [==============================] - 35s 3s/step - loss: 0.8133 - binary_accuracy: 0.5000 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 4/10
12/12 [==============================] - 34s 3s/step - loss: 0.8133 - binary_accuracy: 0.5000 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 5/10
12/12 [==============================] - 36s 3s/step - loss: 0.9799 - binary_accuracy: 0.3333 - val_loss: 0.8133 - v

In [5]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)





Epoch 1/10
12/12 [==============================] - 159s 13s/step - loss: 0.8133 - binary_accuracy: 0.5000 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 2/10
12/12 [==============================] - 161s 13s/step - loss: 0.8966 - binary_accuracy: 0.4167 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 3/10
12/12 [==============================] - 162s 14s/step - loss: 0.9799 - binary_accuracy: 0.3333 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 4/10
12/12 [==============================] - 178s 15s/step - loss: 0.9799 - binary_accuracy: 0.3333 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 5/10
12/12 [==============================] - 167s 14s/step - loss: 0.9799 - binary_accuracy: 0.3333 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 6/10
12/12 [==============================] - 162s 14s/step - loss: 0.8966 - binary_accuracy: 0.4167 - val_loss: 0.8133 - val_binary_accuracy: 0.5000
Epoch 7/10
12/12 [==============================] - 162s 13s/ste

NameError: name 'X_test' is not defined